In [49]:
import numpy as np 
import pandas as pd

In [50]:
def CW_PF(Re, D, ks):

    # Paraḿetros de solver
    err = 1e10
    tol = 1e-10
    count = 1
    f0 = 0.02

    # Iterar
    while err > tol:

        # Calcular el f nuevo
        f1 = (2 * np.log10(ks / D / 3.7 + 2.51 / Re / np.sqrt(f0))) ** -2

        # Calculando error
        err = np.abs((f1 - f0) / f0)

        # Imprimir resultados
        print(f'Iteración: {count}\tf = {f1:.7f}\tError: {err:.6f}')

        # Actualizo valores
        f0 = f1
        count += 1

    return f1

# Hallar f por Newton-Raphson
def CW_NR(Re, D, ks):

    # Paraḿetros de solver
    err = 1e10
    tol = 1e-10
    # count = 1
    x0 = 0.02 ** -2
    K = 2 * 2.51 / np.log(10)

    gx = lambda x, Re, D, ks: x + 2 * np.log10(ks / D / 3.7 + 2.51 * x/ Re)
    gpx = lambda x, Re, D, ks: 1 + K / (ks / D / 3.7 + 2.51 * x / Re) / Re

    # Iterando 
    while err > tol: 

        # Calcular x1
        x1 = x0 - gx(x0, Re, D, ks) / gpx(x0, Re, D, ks)

        # Calculando error
        err = np.abs((x1 - x0) / x0)

        # print(f'Iteración: {count}\tf = {x1 ** -2:.7f}\tErr: {err:.7f}')

        # count += 1
        x0 = x1

    return 1 / x0 ** 2

In [51]:
print('Con Punto fijo:')
CW_PF(150000, 150, 0.0015)

print('Con Newton:')
CW_NR(150000, 150, 0.0015)

Con Punto fijo:
Iteración: 1	f = 0.0162932	Error: 0.185342
Iteración: 2	f = 0.0166617	Error: 0.022621
Iteración: 3	f = 0.0166209	Error: 0.002452
Iteración: 4	f = 0.0166253	Error: 0.000269
Iteración: 5	f = 0.0166248	Error: 0.000030
Iteración: 6	f = 0.0166249	Error: 0.000003
Iteración: 7	f = 0.0166249	Error: 0.000000
Iteración: 8	f = 0.0166249	Error: 0.000000
Iteración: 9	f = 0.0166249	Error: 0.000000
Iteración: 10	f = 0.0166249	Error: 0.000000
Iteración: 11	f = 0.0166249	Error: 0.000000
Con Newton:


0.016624898330623673

In [52]:
# Problema de chequeo de diseño de tubería simple: 
Di = 0.293                              # D int (m)
L = 730                                 # Long (m)
Dz = 43.5                               # Pérdida tot (m)
ks = 1.5e-6                             # Rugosidad (m)
SK = 11.8                               # Suma accesorios
nu = 1.007e-6                           # Visco cinematic (m2/s)
f0 = 0.025                              # Supuesto f

# Parámetros numéricos
err = 1e10
tol = 1e-12
count = 1

# Vectores para resultados
vf = []
vv = []
vRe = []

while err > tol: 

    # Calcular v
    v = np.sqrt((19.62 * Dz) / (f0 * L / Di + SK))

    # Estimar Re
    Re = v * Di / nu

    # Estimo nuevo valor de f
    f1 = CW_NR(Re, Di, ks)

    # Calcular error
    err = np.abs((f1 - f0) / f0)

    print(f'Iteración: {count}\tv: {v:.4}\tf={f1:.6}')

    vf.append(f0)
    vv.append(v)
    vRe.append(Re)


    count += 1
    f0 = f1

Iteración: 1	v: 3.394	f=0.0117859
Iteración: 2	v: 4.553	f=0.0112428
Iteración: 3	v: 4.63	f=0.0112133
Iteración: 4	v: 4.634	f=0.0112117
Iteración: 5	v: 4.635	f=0.0112116
Iteración: 6	v: 4.635	f=0.0112116
Iteración: 7	v: 4.635	f=0.0112116
Iteración: 8	v: 4.635	f=0.0112116
Iteración: 9	v: 4.635	f=0.0112116
Iteración: 10	v: 4.635	f=0.0112116
Iteración: 11	v: 4.635	f=0.0112116


In [57]:
Datos = pd.DataFrame({
    'f': np.round(vf, 6), 
    'v (m/s)': np.round(vv, 4),
    'Re': np.round(vRe, 2)
})

Datos['Q (m3/s)'] = np.round(Datos['v (m/s)'] * np.pi * Di ** 2 / 4, 4)
Datos['Q (L/s)'] = Datos['Q (m3/s)'] * 1000
Datos['hf (m)'] = np.round((Datos['f'] * L / Di) * Datos['v (m/s)'] ** 2 / 19.62, 4)
Datos['hk (m)'] = np.round(SK * Datos['v (m/s)'] ** 2 / 19.62, 4)
Datos['HT (m)'] = Datos['hf (m)'] + Datos['hk (m)']

Datos

,f,v (m/s),Re,Q (m3/s),Q (L/s),hf (m),hk (m),HT (m)
0,0.025000,3.3941,987557.03,0.2288,228.8,36.5717,6.9284,43.5001
1,0.011786,4.5534,1324869.01,0.3070,307.0,31.0309,12.4697,43.5006
2,0.011243,4.6301,1347192.68,0.3122,312.2,30.6069,12.8933,43.5002
3,0.011213,4.6344,1348438.63,0.3125,312.5,30.5819,12.9172,43.4991
4,0.011212,4.6346,1348507.48,0.3125,312.5,30.5819,12.9184,43.5003
5,0.011212,4.6346,1348511.29,0.3125,312.5,30.5819,12.9184,43.5003
6,0.011212,4.6346,1348511.50,0.3125,312.5,30.5819,12.9184,43.5003
7,0.011212,4.6346,1348511.51,0.3125,312.5,30.5819,12.9184,43.5003
8,0.011212,4.6346,1348511.51,0.3125,312.5,30.5819,12.9184,43.5003
9,0.011212,4.6346,1348511.51,0.3125,312.5,30.5819,12.9184,43.5003
